# Make mutation annotations that include nucleotide changes to codon required for each mutation

In [ ]:
import os

import Bio.Seq
import Bio.SeqIO

import pandas as pd

For every codon, get the number of nucleotide changes needed to get to any other amino acid:

In [ ]:
codons = [f"{nt1}{nt2}{nt3}" for nt1 in "ACGT" for nt2 in "ACGT" for nt3 in "ACGT"]

distances = {}
for codon1 in codons:
    distances[(codon1, "-")] = 3
    for codon2 in codons:
        aa2 = str(Bio.Seq.Seq(codon2).translate())
        nt_diffs = sum(x != y for (x, y) in zip(codon1, codon2))
        if (codon1, aa2) in distances:
            distances[(codon1, aa2)] = min(distances[(codon1, aa2)], nt_diffs)
        else:
            distances[(codon1, aa2)] = nt_diffs

distances = pd.DataFrame(
    [(codon1, aa2, nt_diffs) for ((codon1, aa2), nt_diffs) in distances.items()],
    columns=["wildtype_codon", "mutant", "nt changes to codon"]
)

distances

Now read the wildtype gene sequence, the site numbering map, and the PacBio amplicon.
Make sure the wildtype gene sequence encodes the same protein as the PacBio amplicon:

In [ ]:
natural_geneseq = Bio.SeqIO.read(snakemake.input.natural_geneseq, "fasta")
if len(natural_geneseq) % 3:
    raise ValueError(f"Invalid {len(natural_geneseq)=}")
natural_geneseq_str = str(natural_geneseq.seq).upper()

pacbio_amplicon = Bio.SeqIO.read(snakemake.input.pacbio_amplicon, "genbank")
pacbio_gene_feature = [feat for feat in pacbio_amplicon.features if feat.type == "gene"]
assert len(pacbio_gene_feature) == 1
pacbio_gene_feature = pacbio_gene_feature[0]
pacbio_gene = pacbio_gene_feature.extract(pacbio_amplicon)

# check natural_geneseq and pacbio_gene encode same protein
natural_prot = str(natural_geneseq.translate().seq)
if natural_prot.endswith("*"):
    natural_prot = natural_prot[: -1]
pacbio_prot = str(pacbio_gene.translate().seq)
if pacbio_prot.endswith("*"):
    pacbio_prot = pacbio_prot[: -1]
if natural_prot != pacbio_prot:
    raise ValueError(f"difference between {pacbio_prot=} and {natural_prot=}")

site_numbering_map = pd.read_csv(snakemake.input.site_numbering_map)[
    ["sequential_site", "reference_site"]
]

if len(site_numbering_map) * 3 != len(natural_geneseq):
    raise ValueError(f"{len(site_numbering_map)=} and {len(natural_geneseq)=}")
assert list(range(1, len(site_numbering_map) + 1)) == site_numbering_map["sequential_site"].tolist()

mutation_annotations = (
    site_numbering_map
    .assign(
        wildtype_codon=lambda x: x["sequential_site"].map(
            lambda r: natural_geneseq_str[3 * (r - 1): 3 * r]
        ),
    )
    .merge(distances, on="wildtype_codon", how="left")
    .sort_values(["sequential_site", "mutant"])
    .rename(columns={"reference_site": "site"})
    .drop(columns=["sequential_site", "wildtype_codon"])
)

assert len(mutation_annotations) == len(site_numbering_map) * 22

print(f"Writing the mutation annotations to {snakemake.output.annotations}")
os.makedirs(os.path.dirname(snakemake.output.annotations), exist_ok=True)
mutation_annotations.to_csv(snakemake.output.annotations, index=False)

mutation_annotations